## *1. CLEANING WORLD HAPPINESS REPORT DATA* ##

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

# Loading data and adding year column
whr_2015 = pd.read_csv('../Raw/dataset2_world_happiness_report/2015.csv')
whr_2015['Year'] = 2015

whr_2016 = pd.read_csv('../Raw/dataset2_world_happiness_report/2016.csv')
whr_2016['Year'] = 2016

whr_2017 = pd.read_csv('../Raw/dataset2_world_happiness_report/2017.csv')
whr_2017['Year'] = 2017

whr_2018 = pd.read_csv('../Raw/dataset2_world_happiness_report/2018.csv')
whr_2018['Year'] = 2018

whr_2019 = pd.read_csv('../Raw/dataset2_world_happiness_report/2019.csv')  # Fixed file name
whr_2019['Year'] = 2019

In [ ]:
d1 = pd.read_csv('../Raw/dataset1_avg_annual_hours_worked.csv')

In [ ]:
#renaming columns to common names

whr_2015.rename(columns={
    "Year":"year",
    "Country":"country",
    "Region":"region",
    "Happiness Score": "happiness",
    "Economy (GDP per Capita)": "gdp_per_capita",
    "Family": "social_support",
    "Health (Life Expectancy)": "life_expectancy",
    "Freedom": "freedom",
    "Generosity": "generosity",
    "Trust (Government Corruption)": "govt_corruption"
}, inplace=True)

whr_2016.rename(columns={
    "Year":"year",
    "Country":"country",
    "Region":"region",
    "Happiness Score": "happiness",
    "Economy (GDP per Capita)": "gdp_per_capita",
    "Family": "social_support",
    "Health (Life Expectancy)": "life_expectancy",
    "Freedom": "freedom",
    "Generosity": "generosity",
    "Trust (Government Corruption)": "govt_corruption"
}, inplace=True)

whr_2017.rename(columns={
    "Year":"year",
    "Country":"country",
    "Happiness.Score": "happiness",
    "Economy..GDP.per.Capita.": "gdp_per_capita",
    "Family": "social_support",
    "Health..Life.Expectancy.": "life_expectancy",
    "Freedom": "freedom",
    "Generosity": "generosity",
    "Trust..Government.Corruption.": "govt_corruption"
}, inplace=True)

whr_2018.rename(columns={
    "Year":"year",
    "Country or region":"country",
    "Score": "happiness",
    "GDP per capita": "gdp_per_capita",
    "Social support": "social_support",
    "Healthy life expectancy": "life_expectancy",
    "Freedom to make life choices": "freedom",
    "Generosity": "generosity",
    "Perceptions of corruption": "govt_corruption"
}, inplace=True)

whr_2019.rename(columns={
    "Year":"year",
    "Country or region":"country",
    "Score": "happiness",
    "GDP per capita": "gdp_per_capita",
    "Social support": "social_support",
    "Healthy life expectancy": "life_expectancy",
    "Freedom to make life choices": "freedom",
    "Generosity": "generosity",
    "Perceptions of corruption": "govt_corruption"
}, inplace=True)


In [ ]:
#merging the different dataframes

whr_long = pd.concat([whr_2015, whr_2016, whr_2017, whr_2018, whr_2019], ignore_index=True)

columns_to_keep = ["country", "region", "happiness", "gdp_per_capita", "social_support", "life_expectancy", "freedom", "govt_corruption", "generosity", "year"]

whr_combined = whr_long[columns_to_keep]

In [ ]:
#reordering year col

year_col = whr_combined.pop("year")  # Remove 'Year' column
whr_combined.insert(0, "year", year_col) 

In [ ]:
#Fill missing 'Region' values with region values based on country column in other years
whr_combined["region"] = whr_combined.groupby("country")["region"].ffill().bfill()

In [ ]:
#checking for null values
print(whr_combined.isnull().sum())

In [ ]:
#compute the minimum non-null value of 'govt_corruption'
min_value = whr_combined["govt_corruption"].min()

#fill missing values with the computed minimum
whr_combined["govt_corruption"].fillna(min_value, inplace=True)

In [ ]:
whr_combined

## *2. CLEANING AVG HOURS WORKED PER PERSON PER YEAR* ##

In [ ]:
#changing columns to lowercase
d1.columns = d1.columns.str.lower()

#removing wide spaces
d1.columns = d1.columns.str.replace(' ','_')

#removing unwanted columns
d1.drop(['structure', 'structure_name', 'action', 'ref_area','measure',	
         'measure',	'unit_measure',	'unit_of_measure',
        'obs_status', 'observation_status',	'unit_mult', 'unit_multiplier',	'decimals',	'decimals',
        'sex', 'sex', 'age', 'age',	'labour_force_status',	'labour_force_status',	'work_period',	'work_period', 'aggregation_operation',	
         'aggregation_operation',	'hour_bands', 'hour_bands', 'job_coverage','job_coverage',
        'hours_type', 'hours_type',	'worker_status', 'worker_status', 'work_time_arngmnt',	'working_time_arrangement',
        'structure_id','observation_value'], axis=1, inplace=True)

#removing the second value time_period column, since its  duplicated
d1 = d1.iloc[:, ~d1.columns.duplicated()]

# Reset the index, turn it into a column
d1 = d1.reset_index()
whr_combined = whr_combined.reset_index()

whr_combined = whr_combined.reset_index()
# Rename the index column properly
d1 = d1.rename(columns={'index': 'hours_id', 'reference_area': 'country', 'obs_value': 'average_hours', 'time_period': 'year'})
whr_combined = whr_combined.rename(columns ={'index': 'whr_id'})


In [ ]:
display(d1)
display(whr_combined)

## *3. CREATE JUNCTION TABLE* ##

In [ ]:
# set index=false when exporting dataframe .replace("\n", " ")
# df[col] = df[col].apply(lambda s: s.replace("\n"," ") )
# df.to_csv(filename, index=False, sep=";", encoding="utf-8")

In [ ]:
print("Central African\nRepublic".replace("\n"," "))

In [ ]:
countries = whr_combined[['country']].drop_duplicates().reset_index(drop=True)

countries['country_id'] = countries.index

In [ ]:
#remove
# Congo (Brazzaville) - whr_combined
# Congo (Kinshasa) - whr_combined



# Remove Hong Kong - d1

# Remove OECD - d1

# Remove Palestinian Territories - whr_combined

# remove - Somalia - Somalia - Somaliland region - Somaliland Region - whr_combined

# remove Taiwan = Taiwan Province of China - whr_combined

# remove Trinidad & Tobago - Trinidad and Tobago - whr_combined


In [ ]:
whr_combined['country'].replace("Czech Republic", "Czechia", inplace=True)
whr_combined[whr_combined['country'] == 'Czechia']

In [ ]:
d1['country'].replace("Türkiye", "Turkey", inplace=True)
d1[d1['country'] == 'Turkey']

In [ ]:
d1['country'].replace("Slovak Republic", "Slovakia", inplace=True)
d1[d1['country'] == 'Slovakia']

In [ ]:
#removing some values in country column
whr_combined['country'] = whr_combined['country'].replace('somalia', np.nan, inplace=True)
whr_combined['country'] = whr_combined['country'].replace('somaliland region', np.nan, inplace=True)
whr_combined['country'] = whr_combined['country'].replace('somaliland Region', np.nan, inplace=True)
whr_combined['country'] = whr_combined['country'].replace('Taiwan', np.nan, inplace=True)
whr_combined['country'] = whr_combined['country'].replace('Taiwan province of China', np.nan, inplace=True)
whr_combined['country'] = whr_combined['country'].replace('Trinidad & Tobago', np.nan, inplace=True)
whr_combined['country'] = whr_combined['country'].replace('Trinidad and Tobago', np.nan, inplace)
whr_combined['country'] = whr_combined['country'].replace('Palestinian Territories', np.nan, inplace=True)
whr_combined['country'] = whr_combined['country'].replace('Congo (Brazzaville)', np.nan, inplace=True)
whr_combined['country'] = whr_combined['country'].replace(' Congo (Kinshasa)', np.nan, inplace=True)

whr_combined = whr_combined.dropna(subset=['country'])

In [ ]:
#remving unwanted values from d1 table 
d1['country'] = d1['country'].replace('Hong Kong', np.nan, inplace=True)
d1['country'] = d1['country'].replace('OECD', np.nan, inplace=True)

d1 = d1.dropna(subset=['country'])

In [ ]:
regions = whr_combined[['region']].drop_duplicates().reset_index(drop=True)

regions['region_id'] = regions.index

In [ ]:
regions